# Champlin Smith Milestone #5

In [78]:
import sqlite3

def get_connection(db_name="project_database.db"):
    conn = sqlite3.connect(db_name)
    return conn


In [ ]:
import pandas as pd

df = pd.read_csv('C:\\Users\\champ\\OneDrive\\Documents\\MS Data Science\\DSC680 Applied Data Science\\DSC680\\CDCBMI\\ACSST1Y2023.S1501-2025-06-01T175425.csv')
conn = sqlite3.connect('project_database.db')
df.to_sql('education_data', conn, if_exists='replace', index=False)


<>:3: SyntaxWarning: invalid escape sequence '\O'
<>:3: SyntaxWarning: invalid escape sequence '\O'
C:\Users\champ\AppData\Local\Temp\ipykernel_1456\2086401931.py:3: SyntaxWarning: invalid escape sequence '\O'
  df = pd.read_csv('C:\\Users\\champ\OneDrive\\Documents\\MS Data Science\\DSC540 Data Preparation\\ACSST1Y2023.S1501-2025-06-01T175425.csv')


52

In [80]:
def print_all_education_data(db_name="education.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM education_data")
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    conn.close()

# Call the function to print all rows
print_all_education_data()

('Alabama', '479,855', '51,796', '182,028', '194,385', '51,646', '3,498,967', '121,630', '258,272', '1,052,554', '733,923', '321,748', '616,025', '394,815', '3,119,065', '1,010,840', '644,517', '597,617', '201,926', '649,252', '581,414', '211,253', '1,273,079', '1,123,198', '358,913', '932,119', '816,836', '238,748', '45,781', '29,479', '36,615', '41,938', '60,156', '70,682')
('Alaska', '65,148', '8,523', '33,859', '19,964', '2,802', '493,892', '11,376', '22,085', '143,607', '115,592', '42,025', '99,091', '60,116', '460,431', '159,207', '111,850', '103,350', '31,042', '110,131', '104,675', '41,692', '168,913', '158,248', '54,119', '102,998', '94,158', '32,354', '53,103', '35,109', '43,086', '49,114', '67,221', '86,402')
('Arizona', '701,011', '97,858', '258,209', '265,769', '79,175', '5,150,254', '238,234', '308,931', '1,202,651', '1,203,684', '470,638', '1,051,136', '674,980', '4,603,089', '1,726,116', '1,026,988', '946,943', '345,051', '942,328', '837,677', '335,889', '1,743,207', '1

In [ ]:
df = pd.read_csv('C:\\Users\\champ\\OneDrive\\Documents\\MS Data Science\\DSC680 Applied Data Science\\DSC680\\CDCBMI\\cleaned_population_data.csv')
conn = sqlite3.connect('project_database.db')
df.to_sql('population_data', conn, if_exists='replace', index=False)


<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
C:\Users\champ\AppData\Local\Temp\ipykernel_1456\3310215675.py:1: SyntaxWarning: invalid escape sequence '\O'
  df = pd.read_csv('C:\\Users\\champ\OneDrive\\Documents\\MS Data Science\\DSC540 Data Preparation\\cleaned_population_data.csv')


52

Cleaning the last data set

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('C:\\Users\\champ\\Downloads\\cdc_npao.csv')  # Replace with your actual file name

df = df.dropna(subset=['Sample_Size', 'YearStart'])

# Convert to numeric
df['Sample_Size'] = pd.to_numeric(df['Sample_Size'], errors='coerce')
df['YearStart'] = pd.to_numeric(df['YearStart'], errors='coerce')

# Drop rows where conversion failed
df = df.dropna(subset=['Sample_Size', 'YearStart'])

# Step 1: Filter for the most recent YearStart per LocationDesc
latest_year = df.loc[df.groupby('LocationDesc')['YearStart'].idxmax()]

# Step 2: From that filtered set, select the row with the highest Sample_Size per LocationDesc
final = latest_year.loc[latest_year.groupby('LocationDesc')['Sample_Size'].idxmax()]

# Optional: Reset index
final = final.reset_index(drop=True)

# Save to new CSV
final.to_csv('C:\\Users\\champ\\OneDrive\\Documents\\MS Data Science\\DSC680 Applied Data Science\\DSC680\\CDCBMI\\filtered_by_year_then_sample.csv', index=False)

print("Saved to 'filtered_by_year_then_sample.csv'")

Saved to 'filtered_by_year_then_sample.csv'


In [ ]:
df = pd.read_csv('C:\\Users\\champ\\OneDrive\\Documents\\MS Data Science\\DSC680 Applied Data Science\\DSC680\\CDCBMI\\filtered_by_year_then_sample.csv')
conn = sqlite3.connect('project_database.db')
df.to_sql('weight_data', conn, if_exists='replace', index=False)


<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
C:\Users\champ\AppData\Local\Temp\ipykernel_1456\71489136.py:1: SyntaxWarning: invalid escape sequence '\O'
  df = pd.read_csv('C:\\Users\\champ\OneDrive\\Documents\\MS Data Science\\DSC540 Data Preparation\\filtered_by_year_then_sample.csv')


54

Merging data

In [ ]:
import sqlite3
import pandas as pd

# Paths to your databases
source_db = 'project_database.db'
output_db = 'final_project_merged_data.db'

# Connect to the source database
conn_source = sqlite3.connect(source_db)

# Read the joined data into a pandas DataFrame using LEFT JOIN on 'Area'
query = '''
SELECT e.*, p.*, weight_data.*
FROM education_data e
LEFT JOIN population_data p ON e.Area = p.State
LEFT JOIN weight_data ON e.Area = weight_data.LocationDesc
'''

df_merged = pd.read_sql_query(query, conn_source)
conn_source.close()

# Connect to the new database (this will create the file if it doesn't exist)
conn_output = sqlite3.connect(output_db)

# Write the merged dataframe to a new table called 'merged_data'
df_merged.to_sql('merged_data', conn_output, if_exists='replace', index=False)

conn_output.close()

print(f"Left join completed and saved to {output_db}")


Left join completed and saved to final_project_merged_data.db


In [88]:
merged_db = 'final_project_merged_data.db'

# Connect to the merged database
conn = sqlite3.connect(merged_db)

# Read the merged table into a DataFrame
df = pd.read_sql_query("SELECT * FROM merged_data", conn)

conn.close()

# Save DataFrame to Excel file
excel_file = 'final_merged_data.xlsx'
df.to_excel(excel_file, index=False)

print(f"Data exported successfully to {excel_file}")

Data exported successfully to final_merged_data.xlsx


In [98]:
output = pd.read_excel(excel_file)
print(output)

                    Area Population 18 to 24 years  \
0                Alabama                   479,855   
1                 Alaska                    65,148   
2                Arizona                   701,011   
3               Arkansas                   288,490   
4             California                 3,558,215   
5               Colorado                   531,807   
6            Connecticut                   334,630   
7               Delaware                    88,538   
8   District of Columbia                    69,527   
9                Florida                 1,809,840   
10               Georgia                 1,046,760   
11                Hawaii                   116,354   
12                 Idaho                   192,631   
13              Illinois                 1,137,066   
14               Indiana                   666,323   
15                  Iowa                   315,032   
16                Kansas                   297,531   
17              Kentucky    